In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
 ! pip3 install s3fs 

     |████████████████████████████████| 118 kB 43.4 MB/s 
     |████████████████████████████████| 50 kB 6.9 MB/s 
     |████████████████████████████████| 7.7 MB 33.7 MB/s 
     |████████████████████████████████| 1.3 MB 49.9 MB/s 
     |████████████████████████████████| 142 kB 60.1 MB/s 
     |████████████████████████████████| 294 kB 42.4 MB/s 
     |████████████████████████████████| 138 kB 61.5 MB/s 
  Created wheel for aiobotocore: filename=aiobotocore-1.3.3-py3-none-any.whl size=48358 sha256=840fbf00e5a1779ec5274618609e9f502fbc0e9b3552899c206153742728fe41
  Stored in directory: /root/.cache/pip/wheels/29/82/8c/1fdaf3716b8ed7b531866c66f388efdaa35c38ae48b0fd9415
Successfully built aiobotocore
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of 

In [3]:
!pip3 install --upgrade gspread

     |████████████████████████████████| 127 kB 40.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.6
    Uninstalling urllib3-1.26.6:
      Successfully uninstalled urllib3-1.26.6
  Attempting uninstall: gspread
    Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
import pandas as pd
import s3fs
import gzip

In [5]:
import gc
import time

In [6]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gs = gspread.authorize(GoogleCredentials.get_application_default())

In [24]:
# sh = gs.create('RDP SafeGraph WP NY QA')
# worksheet = sh.add_worksheet(title="YZ", rows="1000", cols="20")
sh = gs.open("RDP SafeGraph WP NY QA")
wks_yz = sh.worksheet('YZ')
wks_jt = sh.worksheet('JT')

In [13]:
s3 = s3fs.S3FileSystem(
      key='xxx',
      secret='xxx', 
      client_kwargs={
          'endpoint_url': 'https://s3.amazonaws.com', 
          'region_name':'us-east-1'
      }
    )

In [19]:
def get_data(path,n=10):
  with s3.open(path, mode='rb') as f:
    if path[-2:] == 'gz':
      f = gzip.GzipFile(fileobj=f)
    return pd.read_csv(f, nrows=n)

In [10]:
s3.ls('recovery-data-partnership/output/dev/wp_NYC')

['recovery-data-partnership/output/dev/wp_NYC/weekly_patterns_NYC_2019Q1.csv.zip',
 'recovery-data-partnership/output/dev/wp_NYC/weekly_patterns_NYC_2019Q2.csv.zip',
 'recovery-data-partnership/output/dev/wp_NYC/weekly_patterns_NYC_2019Q3.csv.zip',
 'recovery-data-partnership/output/dev/wp_NYC/weekly_patterns_NYC_2019Q4.csv.zip',
 'recovery-data-partnership/output/dev/wp_NYC/weekly_patterns_NYC_2020Q1.csv.zip',
 'recovery-data-partnership/output/dev/wp_NYC/weekly_patterns_NYC_2020Q2.csv.zip',
 'recovery-data-partnership/output/dev/wp_NYC/weekly_patterns_NYC_2020Q3.csv.zip',
 'recovery-data-partnership/output/dev/wp_NYC/weekly_patterns_NYC_2020Q4.csv.zip',
 'recovery-data-partnership/output/dev/wp_NYC/weekly_patterns_NYC_2021Q1.csv.zip',
 'recovery-data-partnership/output/dev/wp_NYC/weekly_patterns_NYC_2021Q2.csv.zip',
 'recovery-data-partnership/output/dev/wp_NYC/weekly_patterns_NYC_2021Q3.csv.zip']

In [11]:
for f in s3.ls('recovery-data-partnership/output/dev/wp_NYC'):
  f = f.split('/')
  print(f[-1])

weekly_patterns_NYC_2019Q1.csv.zip
weekly_patterns_NYC_2019Q2.csv.zip
weekly_patterns_NYC_2019Q3.csv.zip
weekly_patterns_NYC_2019Q4.csv.zip
weekly_patterns_NYC_2020Q1.csv.zip
weekly_patterns_NYC_2020Q2.csv.zip
weekly_patterns_NYC_2020Q3.csv.zip
weekly_patterns_NYC_2020Q4.csv.zip
weekly_patterns_NYC_2021Q1.csv.zip
weekly_patterns_NYC_2021Q2.csv.zip
weekly_patterns_NYC_2021Q3.csv.zip


In [16]:
yz_counts = {}
for path in s3.ls('recovery-data-partnership/output/dev/wp_NYC'):
  start_time = time.time()
  with s3.open(path, mode='rb') as f:
    df = pd.read_csv(f,compression='zip')
  file_name = path.split('/')[-1]
  print(f"filename: {file_name} contains {len(df)} obs")
  date_summary = df['date_range_start'].str[:10].value_counts(dropna=False).sort_index().reset_index()
  print(date_summary)
  del df
  gc.collect()
  duration = time.time() - start_time
  yz_counts[file_name] = date_summary
  print(f"this step took {round(duration/60,1)} minutes")
  print()

filename: weekly_patterns_NYC_2019Q1.csv.zip contains 1005519 obs
         index  date_range_start
0   2019-01-07             83696
1   2019-01-14             83917
2   2019-01-21             83868
3   2019-01-28             83828
4   2019-02-04             83271
5   2019-02-11             84089
6   2019-02-18             83550
7   2019-02-25             83742
8   2019-03-04             83678
9   2019-03-11             83892
10  2019-03-18             83897
11  2019-03-25             84091
this step took 0.9 minutes

filename: weekly_patterns_NYC_2019Q2.csv.zip contains 1092752 obs
         index  date_range_start
0   2019-04-01             84079
1   2019-04-08             84423
2   2019-04-15             84577
3   2019-04-22             83958
4   2019-04-29             84129
5   2019-05-06             84195
6   2019-05-13             84374
7   2019-05-20             83760
8   2019-05-27             83836
9   2019-06-03             83817
10  2019-06-10             83827
11  2019-06-17 

In [17]:
yz_df = pd.DataFrame()
for df in yz_counts:
  yz_df = yz_df.append(yz_counts[df])
yz_df.shape

(134, 2)

In [3]:
# worksheet.update([yz_df.columns.values.tolist()] + yz_df.values.tolist())

In [17]:
s3.ls('recovery-data-partnership/output/release_2021_07/NY_visits_by_month/from_weekly_patterns')

['recovery-data-partnership/output/release_2021_07/NY_visits_by_month/from_weekly_patterns/',
 'recovery-data-partnership/output/release_2021_07/NY_visits_by_month/from_weekly_patterns/NY_weekly_2019_01.csv.gz',
 'recovery-data-partnership/output/release_2021_07/NY_visits_by_month/from_weekly_patterns/NY_weekly_2019_02.csv.gz',
 'recovery-data-partnership/output/release_2021_07/NY_visits_by_month/from_weekly_patterns/NY_weekly_2019_03.csv.gz',
 'recovery-data-partnership/output/release_2021_07/NY_visits_by_month/from_weekly_patterns/NY_weekly_2019_04.csv.gz',
 'recovery-data-partnership/output/release_2021_07/NY_visits_by_month/from_weekly_patterns/NY_weekly_2019_05.csv.gz',
 'recovery-data-partnership/output/release_2021_07/NY_visits_by_month/from_weekly_patterns/NY_weekly_2019_06.csv.gz',
 'recovery-data-partnership/output/release_2021_07/NY_visits_by_month/from_weekly_patterns/NY_weekly_2019_07.csv.gz',
 'recovery-data-partnership/output/release_2021_07/NY_visits_by_month/from_weekl

In [20]:
jt_counts = {}
for path in s3.ls('recovery-data-partnership/output/release_2021_07/NY_visits_by_month/from_weekly_patterns')[1:]:
  start_time = time.time()
  df = get_data(path,n=10000000)
  df_nyc = df[df['poi_cbg'].astype(str).str[:5].isin(['36085','36081','36061','36047','36005'])]
  file_name = path.split('/')[-1]
  print(f"filename: {file_name} contains {len(df)} obs")
  print(f"nyc contains {len(df_nyc)} obs")
  date_summary = df['date_range_start'].str[:10].value_counts(dropna=False).sort_index().reset_index()
  date_nyc_summary = df_nyc['date_range_start'].str[:10].value_counts(dropna=False).sort_index().reset_index()
  full_summary = date_summary.merge(date_nyc_summary,on='index')
  full_summary.columns = ['date_range_start','ny','nyc']
  del df, df_nyc
  gc.collect()
  duration = time.time() - start_time
  jt_counts[file_name] = full_summary
  print(f"this step took {round(duration/60,1)} minutes")
  print()

filename: NY_weekly_2019_01.csv.gz contains 880392 obs
nyc contains 335309 obs
this step took 0.8 minutes

filename: NY_weekly_2019_02.csv.gz contains 879566 obs
nyc contains 334652 obs
this step took 0.8 minutes

filename: NY_weekly_2019_03.csv.gz contains 885096 obs
nyc contains 335558 obs
this step took 0.8 minutes

filename: NY_weekly_2019_04.csv.gz contains 1118607 obs
nyc contains 421166 obs
this step took 1.0 minutes

filename: NY_weekly_2019_05.csv.gz contains 900526 obs
nyc contains 336165 obs
this step took 0.8 minutes

filename: NY_weekly_2019_06.csv.gz contains 898018 obs
nyc contains 335421 obs
this step took 0.9 minutes

filename: NY_weekly_2019_07.csv.gz contains 1110472 obs
nyc contains 414715 obs
this step took 1.0 minutes

filename: NY_weekly_2019_08.csv.gz contains 889875 obs
nyc contains 332026 obs
this step took 0.8 minutes

filename: NY_weekly_2019_09.csv.gz contains 1116981 obs
nyc contains 420796 obs
this step took 1.0 minutes

filename: NY_weekly_2019_10.csv.gz

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


filename: NY_weekly_2019_12.csv.gz contains 1101373 obs
nyc contains 415343 obs
this step took 1.0 minutes

filename: NY_weekly_2020_01.csv.gz contains 904964 obs
nyc contains 341166 obs
this step took 0.9 minutes

filename: NY_weekly_2020_02.csv.gz contains 896376 obs
nyc contains 339662 obs
this step took 0.8 minutes

filename: NY_weekly_2020_03.csv.gz contains 1058289 obs
nyc contains 396611 obs
this step took 0.8 minutes

filename: NY_weekly_2020_04.csv.gz contains 793453 obs
nyc contains 292235 obs
this step took 0.5 minutes

filename: NY_weekly_2020_05.csv.gz contains 825379 obs
nyc contains 300933 obs
this step took 0.6 minutes

filename: NY_weekly_2020_06.csv.gz contains 1072756 obs
nyc contains 388772 obs
this step took 0.8 minutes

filename: NY_weekly_2020_07.csv.gz contains 869540 obs
nyc contains 316802 obs
this step took 0.7 minutes

filename: NY_weekly_2020_08.csv.gz contains 1096656 obs
nyc contains 400825 obs
this step took 0.9 minutes

filename: NY_weekly_2020_09.csv.g

In [22]:
jt_counts['NY_weekly_2021_07.csv.gz']

,date_range_start,ny,nyc
0,2021-07-05,218047,79878


In [25]:
wsh_jt2 = sh.add_worksheet(title="JT2", rows="1000", cols="20")

In [26]:
jt_df = pd.DataFrame()
for df in jt_counts:
  jt_df = jt_df.append(jt_counts[df])
jt_df.shape

(131, 3)

In [2]:
# wsh_jt2.update([jt_df.columns.values.tolist()] + jt_df.values.tolist())